# **Proyecto Hackathon The Data Guys**

## Importación y carga

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import time

from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display, HTML
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import silhouette_score, precision_score, recall_score, f1_score, accuracy_score, roc_curve, roc_auc_score
from IPython.display import display

In [4]:
df = pd.read_csv('Online_Retail.csv', encoding='ISO-8859-1')

## Exploración del Dataset

In [ ]:
# Inspeccionar y visualizar datos

# Convertir todos los nombres de columnas a minúsculas
df.columns = df.columns.str.lower()

# Mostrar información general del DataFrame
print("\n 'Exploración del DF'\n")
display(df.info())

# Mostrar el número de valores nulos en cada columna
print("\n 'Valores Nulos'\n")
display(df.isnull().sum())

# Mostrar el número total de filas duplicadas
print("\n 'Valores Duplicados'\n")
display(df.duplicated().sum())

# Mostrar las primeras filas del DataFrame
print("\n\n 'Vista del DataFrame'\n")
display(df.head())

## Preprocesamiento de los datos

In [ ]:
# Manejar valores nulos en 'customer_id'
# Vamos a eliminar las filas sin customer_id, ya que no podemos hacer segmentación sin esta información
df = df.dropna(subset=['customer_id'])

# Verificar valores nulos nuevamente
print("\n Valores nulos actualizados\n")
print(df.isnull().sum())
print("\n")

# Eliminar duplicados
df = df.drop_duplicates()

# Verificar duplicados nuevamente
duplicados = df.duplicated().sum()
print(f"Filas duplicadas después de la eliminación: {duplicados}")

In [ ]:
# Convertir 'invoice_date' a datetime
df['invoice_date'] = pd.to_datetime(df['invoice_date'], format="%d/%m/%Y %H:%M")

# Verificar el tipo de dato de 'invoice_date'
print(df['invoice_date'].dtype)

In [ ]:
df.describe()

In [ ]:
# Eliminar registros con cantidad o precio unitario negativos
df = df[(df['quantity'] > 0) & (df['unit_price'] > 0)]

# Verificar el rango de 'quantity' y 'unit_price'
df[['quantity', 'unit_price']].describe()

Estadísticas de quantity y unit_price:

La cantidad media de artículos por orden es de 13.12, pero con una desviación estándar alta (180.49), lo que indica una gran variabilidad.
El precio unitario medio es de 3.13, pero también con alta variabilidad (desviación estándar de 22.24).
Hay algunos valores extremos, con un máximo de 80,995 para quantity y 8,142.75 para unit_price.

### Análisis RFM

In [ ]:
# Primero, asegurémonos de que 'customer_id' sea de tipo entero
df['customer_id'] = df['customer_id'].astype(int)

# Calcular el valor monetario total por transacción
df['total_price'] = df['quantity'] * df['unit_price']

# Encontrar la fecha más reciente en el dataset
max_date = df['invoice_date'].max()

# Calcular RFM
rfm = df.groupby('customer_id').agg({
    'invoice_date': lambda x: (max_date - x.max()).days,  # Recency
    'invoice_no': 'count',  # Frequency
    'total_price': 'sum'  # Monetary
})

# Renombrar las columnas
rfm.columns = ['recency', 'frequency', 'monetary']

# Asegurarse de que 'monetary' no tenga valores negativos
rfm = rfm[rfm['monetary'] > 0]

# Mostrar las primeras filas del dataframe RFM
display(rfm.head())

# Mostrar estadísticas descriptivas del dataframe RFM
display(rfm.describe())

# Verificar si hay valores nulos en el dataframe RFM
rfm.isnull().sum()


**Estadísticas RFM:**

- Recency: La media es de 91.61 días, con un mínimo de 0 y un máximo de 374 días.
- Frequency: En promedio, los clientes han realizado 90.52 compras, pero hay una gran variabilidad (desviación estándar de 225.51).
- Monetary: El gasto promedio por cliente es de 2,048.69, pero con una desviación estándar muy alta (8,985.23), indicando una gran dispersión en el gasto.

No hay valores nulos en las métricas RFM, lo cual es bueno para el análisis.

In [ ]:
!pip show seaborn

In [ ]:
# Configurar el estilo de las gráficas
plt.style.use('seaborn-v0_8')
sns.set_palette("deep")
# Función para crear histogramas
def plot_distribution(dataframe, column):
    plt.figure(figsize=(10, 6))
    sns.histplot(data=dataframe, x=column, kde=True)
    plt.title(f'\n\nDistribución de {column.capitalize()}\n')
    plt.xlabel(column.capitalize())
    plt.ylabel('Frecuencia')
    plt.show()

# Crear histogramas para cada métrica RFM
for column in ['recency', 'frequency', 'monetary']:
    plot_distribution(rfm, column)

- Distribución de Recency:

La gráfica muestra una distribución asimétrica positiva (cola hacia la derecha).
Hay una alta concentración de clientes con recency baja (cerca de 0), lo que indica que muchos clientes han realizado compras recientemente.
Hay una cola larga hacia la derecha, lo que sugiere que algunos clientes no han comprado en mucho tiempo.

- Distribución de Frequency:

Muestra una distribución extremadamente asimétrica positiva (cola larga a la derecha).
La gran mayoría de los clientes tienen una frecuencia de compra baja (cerca de 0).
Hay unos pocos clientes con frecuencias de compra muy altas (hasta 7k+).


- Distribución de Monetary:

También presenta una distribución muy asimétrica positiva.
La mayoría de los clientes tienen un valor monetario bajo.
Hay algunos clientes con valores monetarios extremadamente altos (hasta 250,000+).

In [ ]:
# Boxplots para cada métrica RFM
plt.figure(figsize=(15, 5))
for i, column in enumerate(['recency', 'frequency', 'monetary'], 1):
    plt.subplot(1, 3, i)
    sns.boxplot(y=rfm[column])
    plt.title(f'Boxplot de {column.capitalize()}\n')
plt.tight_layout()
plt.show()

In [ ]:
# Para cada métrica RFM
for column in ['recency', 'frequency', 'monetary']:
    print(f"\n\nEstadísticas para {column.capitalize()}:\n")
    stats = rfm[column].describe(percentiles=[.25, .5, .75])
    print(stats)
    
    # Cálculo del IQR y límites para outliers
    Q1 = stats['25%']
    Q3 = stats['75%']
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    print(f"Límite inferior para outliers: {lower_bound}")
    print(f"Límite superior para outliers: {upper_bound}")
    print(f"Número de outliers inferiores: {sum(rfm[column] < lower_bound)}")
    print(f"Número de outliers superiores: {sum(rfm[column] > upper_bound)}")

Gracias por proporcionar estos datos detallados. Vamos a analizarlos para cada métrica RFM:

## Recency:
   - La recencia media es de 91.61 días, con una mediana de 50 días.
   - El 75% de los clientes han comprado en los últimos 141 días.
   - Hay 155 clientes (3.57%) considerados outliers superiores, que no han comprado en más de 327 días.
   - La distribución es asimétrica positiva (media > mediana), lo que indica una cola larga hacia la derecha.

## Frequency:
   - La frecuencia media de compra es 90.52, pero la mediana es solo 41, indicando una fuerte asimetría positiva.
   - El 75% de los clientes han realizado 98 o menos compras.
   - Hay 381 clientes (8.78%) considerados outliers superiores, con más de 219.5 compras.
   - El máximo de 7676 compras sugiere la presencia de algunos clientes extremadamente frecuentes.

## Monetary:
   - El valor monetario medio es de 2048.69, pero la mediana es solo 668.57, indicando una fuerte asimetría positiva.
   - El 75% de los clientes han gastado 1660.60 o menos.
   - Hay 425 clientes (9.80%) considerados outliers superiores, gastando más de 3691.77.
   - El máximo de 280,206.02 sugiere la presencia de algunos clientes de muy alto valor.

## Conclusiones y recomendaciones:

- Asimetría: Todas las métricas muestran una fuerte asimetría positiva, especialmente Frequency y Monetary. Esto refuerza la necesidad de una transformación logarítmica antes del clustering.

- **Outliers**: Hay un número significativo de outliers superiores en todas las métricas, especialmente en Frequency y Monetary. Estos representan clientes de alto valor que merecen un análisis y tratamiento especial.

- Segmentación: La gran diferencia entre la media y la mediana en Frequency y Monetary sugiere que una pequeña proporción de clientes contribuye significativamente al negocio. Una segmentación efectiva podría ayudar a identificar y manejar estos grupos de manera diferenciada.

- Estrategias diferenciadas:
   - Para clientes con alta recencia (>327 días), considerar campañas de reactivación.
   - Para los outliers superiores en Frequency y Monetary, desarrollar programas de fidelización y retención especiales.
   - Para la mayoría de los clientes (que están por debajo de la mediana en Frequency y Monetary), diseñar estrategias para aumentar su frecuencia de compra y valor.

- Preparación para clustering:
   - Aplicar transformación logarítmica a Frequency y Monetary.
   - Normalizar todas las variables.
   - Al interpretar los resultados del clustering, prestar especial atención a cómo se agrupan los outliers.

In [ ]:
def identify_outliers(df, column):
    # Calcular Q1 (primer cuartil) y Q3 (tercer cuartil)
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    # Calcular el Rango Intercuartílico (IQR)
    IQR = Q3 - Q1
    # Definir los límites para los outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Identificar los outliers
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Identificar outliers para cada columna RFM
for column in ['recency', 'frequency', 'monetary']:
    outliers, lower, upper = identify_outliers(rfm, column)
    print(f"\nOutliers en {column}:")
    print(f"Número de outliers: {len(outliers)}")
    print(f"Límite inferior: {lower}")
    print(f"Límite superior: {upper}")
    display(outliers.head())

def extreme_outliers(df, column, factor=3):
    # Similar a identify_outliers, pero con un factor más extremo
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - factor * IQR
    upper_bound = Q3 + factor * IQR
    extremes = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return extremes

# Identificar outliers extremos para cada columna RFM
for column in ['recency', 'frequency', 'monetary']:
    extremes = extreme_outliers(rfm, column)
    print(f"\nOutliers extremos en {column}")
    print(f"Número de outliers extremos: {len(extremes)}")
    display(extremes.head())

def plot_distribution_with_without_outliers(df, column):
    # Identificar outliers
    outliers, lower, upper = identify_outliers(df, column)
    # Crear un DataFrame sin outliers
    non_outliers = df[(df[column] >= lower) & (df[column] <= upper)]
    
    # Crear una figura con dos subplots
    plt.figure(figsize=(12, 5))
    
    # Subplot para la distribución con outliers
    plt.subplot(121)
    sns.histplot(df[column], kde=True)
    plt.title(f'Distribución de {column} con outliers')
    
    # Subplot para la distribución sin outliers
    plt.subplot(122)
    sns.histplot(non_outliers[column], kde=True)
    plt.title(f'Distribución de {column} sin outliers')
    
    plt.tight_layout()
    plt.show()

# Visualizar la distribución con y sin outliers para cada columna RFM
for column in ['recency', 'frequency', 'monetary']:
    plot_distribution_with_without_outliers(rfm, column)

### Distribuciones sin Outliers en Frequency y Monetary

In [ ]:
# Función para remover outliers
def remove_outliers(df, columns):
    df_clean = df.copy()
    for column in columns:
        Q1 = df_clean[column].quantile(0.25)
        Q3 = df_clean[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_clean = df_clean[(df_clean[column] >= lower_bound) & (df_clean[column] <= upper_bound)]
    return df_clean

# Actualizar rfm eliminando outliers de 'monetary' y 'frequency'
rfm = remove_outliers(rfm, ['monetary', 'frequency'])

# Imprimir información sobre el nuevo DataFrame
print("\n")
print(f"Tamaño del DataFrame rfm actualizado: {len(rfm)}")
print("\nEstadísticas descriptivas del rfm actualizado:\n")
display(rfm.describe())

plt.figure(figsize=(15, 5))

plt.subplot(131)
sns.histplot(data=rfm, x='recency', kde=True)
plt.title('Distribución de Recency')

plt.subplot(132)
sns.histplot(data=rfm, x='frequency', kde=True)
plt.title('Distribución de Frequency (sin outliers)')

plt.subplot(133)
sns.histplot(data=rfm, x='monetary', kde=True)
plt.title('Distribución de Monetary (sin outliers)')

plt.tight_layout()
plt.show()

In [ ]:
# Scatter plots para visualizar relaciones entre métricas
plt.figure(figsize=(15, 5))
plt.subplot(131)
sns.scatterplot(data=rfm, x='recency', y='frequency')
plt.title('Recency vs Frequency')
plt.subplot(132)
sns.scatterplot(data=rfm, x='recency', y='monetary')
plt.title('Recency vs Monetary')
plt.subplot(133)
sns.scatterplot(data=rfm, x='frequency', y='monetary')
plt.title('Frequency vs Monetary')
plt.tight_layout()
plt.show()

- Recency vs Frequency: Muestra una concentración de puntos en la esquina inferior izquierda, indicando que muchos clientes tienen baja recencia y baja frecuencia. Hay algunos outliers con frecuencia muy alta.
- Recency vs Monetary: Similar al anterior, con la mayoría de los puntos concentrados en valores bajos, pero con algunos outliers de alto valor monetario.
- Frequency vs Monetary: Muestra una relación positiva más clara, con algunos clientes destacando por tener valores muy altos en ambas métricas.

In [ ]:
# Calcular la matriz de correlación
corr_matrix = rfm[['recency', 'frequency', 'monetary']].corr()

print("\nMatriz de correlación:\n")
display(corr_matrix)
print("\n")

# Función para describir la relación entre dos variables
def describe_relationship(x, y, x_name, y_name):
    print(f"Relación entre {x_name} y {y_name}:")
    print("\n")
    print(f"Correlación: {x.corr(y):.4f}")
    print(f"Covarianza: {x.cov(y):.4f}")
    print(f"{x_name} - Media: {x.mean():.4f}, Desviación estándar: {x.std():.4f}")
    print(f"{y_name} - Media: {y.mean():.4f}, Desviación estándar: {y.std():.4f}")
    print("\n")

# Describir las relaciones
describe_relationship(rfm['recency'], rfm['frequency'], 'Recency', 'Frequency')
describe_relationship(rfm['recency'], rfm['monetary'], 'Recency', 'Monetary')
describe_relationship(rfm['frequency'], rfm['monetary'], 'Frequency', 'Monetary')

# Matriz de correlación
plt.figure(figsize=(10, 8))
sns.heatmap(rfm.corr(), annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Matriz de Correlación RFM\n')
plt.show()

**Recency y Frequency:**
- -La correlación es negativa (-0.3514), lo que indica que a medida que la Recency aumenta, la Frequency tiende a disminuir, y viceversa. Esto podría sugerir que los clientes que han comprado más recientemente tienden a hacer compras con menos frecuencia.

**Recency y Monetary:**
- La correlación también es negativa (-0.3275), lo que sugiere que los clientes que han comprado más recientemente tienden a gastar menos, y viceversa. Esto podría indicar que los clientes que gastan más tienden a hacerlo en compras menos frecuentes.

**Frequency y Monetary:**
- La correlación es positiva (0.6733), lo que indica que a medida que la Frequency aumenta, la Monetary también tiende a aumentar. Esto sugiere que los clientes que hacen compras con más frecuencia también tienden a gastar más en total.

Estas son solo observaciones generales y podrían no aplicarse a todos los clientes. También es importante tener en cuenta otros factores que podrían influir en estas relaciones.

In [ ]:
# Estadísticas descriptivas
display(rfm.describe())

# Asimetría y curtosis
print("\nAsimetría:")
print(rfm.skew())
print("\nCurtosis:")
print(rfm.kurtosis())

- La asimetría es una medida de la falta de simetría en la distribución de los datos. Un valor de asimetría **positivo** indica una distribución con cola a la derecha o sesgada a la derecha.

- La curtosis es una medida de la **pesadez** de las colas de una distribución. En comparación con una distribución normal, un valor de curtosis positivo indica colas más pesadas (más propensas a tener valores extremos), mientras que un valor negativo indica colas más ligeras (menos propensas a tener valores extremos).

### Segmentación de clientes (Clustering)

In [ ]:
# Aplicar transformación logarítmica
rfm['log_frequency'] = np.log1p(rfm['frequency'])
rfm['log_monetary'] = np.log1p(rfm['monetary'])

In [ ]:
# Crear un nuevo DataFrame con las variables transformadas
rfm_normalized = rfm[['recency', 'log_frequency', 'log_monetary']].copy()

# Normalizar las variables
scaler = StandardScaler()
rfm_normalized = pd.DataFrame(scaler.fit_transform(rfm_normalized), 
                              columns=['recency', 'log_frequency', 'log_monetary'], 
                              index=rfm.index)

In [ ]:
# Función para calcular la inercia (suma de las distancias al cuadrado)
def calculate_inertia(data, k_range):
    inertias = []
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(data)
        inertias.append(kmeans.inertia_)
    return inertias

# Función para calcular el score de silueta
def calculate_silhouette(data, k_range):
    silhouette_scores = []
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(data)
        score = silhouette_score(data, labels)
        silhouette_scores.append(score)
    return silhouette_scores

# Rango de número de clusters a probar
k_range = range(2, 25)

# Calcular inercia y score de silueta
inertias = calculate_inertia(rfm_normalized, k_range)
silhouette_scores = calculate_silhouette(rfm_normalized, k_range)

# Visualizar el método del codo
plt.figure(figsize=(12, 5))
plt.subplot(121)
plt.plot(k_range, inertias, marker='o')
plt.xlabel('Número de clusters (k)')
plt.ylabel('Inercia')
plt.title('Método del Codo')

plt.subplot(122)
plt.plot(k_range, silhouette_scores, marker='o')
plt.xlabel('Número de clusters (k)')
plt.ylabel('Score de Silueta')
plt.title('Método de la Silueta')

plt.tight_layout()
plt.show()


**Número óptimo de clusters**

El gráfico del método del codo muestra una disminución pronunciada de la inercia hasta aproximadamente 4-5 clusters, después de lo cual la disminución se vuelve más gradual. El gráfico del score de silueta muestra valores más altos para números menores de clusters, con un pico local alrededor de 4-5 clusters. Por lo que estableceremos optimal_k = 5.

In [ ]:
# Número óptimo de clusters
optimal_k = 5

kmeans = KMeans(n_clusters=optimal_k, random_state=42)
rfm['Cluster'] = kmeans.fit_predict(rfm_normalized)

# Añadir los clusters al DataFrame original
rfm_with_clusters = rfm.copy()
rfm_with_clusters['Cluster'] = rfm['Cluster']

In [ ]:
# Estadísticas descriptivas por cluster
cluster_stats = rfm_with_clusters.groupby('Cluster').agg({
    'recency': 'mean',
    'frequency': 'mean',
    'monetary': 'mean'
})

# Añadir el conteo de clientes por cluster
cluster_stats['count'] = rfm_with_clusters.groupby('Cluster').size()

print("\nEstadísticas por cluster\n")
display(cluster_stats)

# Visualizar los clusters
plt.figure(figsize=(12, 8))
scatter = plt.scatter(rfm_normalized['log_frequency'], 
                      rfm_normalized['log_monetary'], 
                      c=rfm['Cluster'], 
                      cmap='viridis')
plt.xlabel('Log Frequency (Normalizado)')
plt.ylabel('Log Monetary (Normalizado)')
plt.title('Clusters de Clientes')
plt.colorbar(scatter)
plt.show()

In [ ]:
# Gráfico 3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(rfm['recency'], rfm['frequency'], rfm['monetary'], 
                     c=rfm['Cluster'], cmap='viridis')

ax.set_xlabel('Recency')
ax.set_ylabel('Frequency')
ax.set_zlabel('Monetary')
plt.title('Visualización 3D de Clusters RFM')
plt.colorbar(scatter)
plt.show()

# Gráficos 2D
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

axes[0].scatter(rfm['recency'], rfm['frequency'], c=rfm['Cluster'], cmap='viridis')
axes[0].set_xlabel('Recency')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Recency vs Frequency')

axes[1].scatter(rfm['recency'], rfm['monetary'], c=rfm['Cluster'], cmap='viridis')
axes[1].set_xlabel('Recency')
axes[1].set_ylabel('Monetary')
axes[1].set_title('Recency vs Monetary')

axes[2].scatter(rfm['frequency'], rfm['monetary'], c=rfm['Cluster'], cmap='viridis')
axes[2].set_xlabel('Frequency')
axes[2].set_ylabel('Monetary')
axes[2].set_title('Frequency vs Monetary')

plt.tight_layout()
plt.show()

### Clientes de alto valor (Cluster 4):
- **Estrategia**: Programas de fidelización, ofertas exclusivas, servicio premium.
- **Objetivo**: Mantener su alta frecuencia de compra y aumentar el valor por compra.

### Clientes activos de valor medio (Cluster 3):
- **Estrategia**: Incentivos para aumentar el valor de compra, cross-selling.
- **Objetivo**: Incrementar el valor monetario de sus compras.

### Clientes recientes de bajo valor (Cluster 2):
- **Estrategia**: Campañas para aumentar la frecuencia de compra, ofertas atractivas.
- **Objetivo**: Convertirlos en clientes más frecuentes y de mayor valor.

### Clientes de valor medio con baja recencia (Cluster 0):
- **Estrategia**: Campañas de reactivación, ofertas personalizadas basadas en compras anteriores.
- **Objetivo**: Reducir el tiempo desde la última compra y aumentar la frecuencia.

### Clientes inactivos de bajo valor (Cluster 1):
- **Estrategia**: Campañas de reactivación agresivas, encuestas para entender la inactividad.
- **Objetivo**: Recuperar estos clientes o decidir si vale la pena invertir en ellos.

**Observaciones generales:**

Los clusters 3 y 4 (clientes activos y de alto valor) representan la mayor parte de la base de clientes (2115 de 3662, o 57.8%).
Hay una clara diferenciación entre los segmentos en términos de comportamiento de compra.
La recencia parece ser un factor importante en la segmentación, con una clara distinción entre clientes activos e inactivos.

In [ ]:
# Asumiendo que tenemos información adicional como 'product_category' y 'purchase_month'
for cluster in rfm['Cluster'].unique():
    cluster_data = rfm[rfm['Cluster'] == cluster]
    
    print(f"\nAnálisis de Cluster {cluster}")
    print(f"\nEstadísticas RFM Cluster {cluster}")
    display(cluster_data[['recency', 'frequency', 'monetary']].describe())
    print("\n" + "="*50)

# Análisis de Clusters RFM

## Resumen de Clusters

| Cluster | Nombre | Recencia (días) | Frecuencia | Monetario ($) | Clientes |
|---------|--------|-----------------|------------|---------------|----------|
| 4 | Clientes de alto valor | 41 | 93.5 | 1669 | 1052 |
| 3 | Clientes activos de valor medio | 48 | 36.7 | 601 | 1063 |
| 2 | Clientes recientes de bajo valor | 67 | 10.5 | 226 | 594 |
| 0 | Clientes de valor medio con baja recencia | 240 | 36.1 | 563 | 549 |
| 1 | Clientes inactivos de bajo valor | 278 | 8.5 | 191 | 404 |

## Análisis Detallado por Cluster

### Cluster 4: Clientes de alto valor
- **Características**: Muy activos, alta frecuencia, alto valor
- **Estrategia**: Retención y maximización de valor
- **Acciones**:
  1. Implementar programa VIP exclusivo
  2. Ofrecer acceso anticipado a nuevos productos
  3. Proporcionar servicio al cliente dedicado

### Cluster 3: Clientes activos de valor medio
- **Características**: Activos, frecuencia media, valor medio
- **Estrategia**: Incremento de valor por compra
- **Acciones**:
  1. Crear programa de escalado de compras
  2. Ofrecer paquetes de productos para aumentar el valor por transacción
  3. Desarrollar contenido educativo sobre productos premium

### Cluster 2: Clientes recientes de bajo valor
- **Características**: Recientes, baja frecuencia, bajo valor
- **Estrategia**: Aumento de frecuencia y valor
- **Acciones**:
  1. Lanzar campaña de "segunda compra" con incentivos
  2. Implementar sistema de recomendaciones personalizadas
  3. Introducir programa de fidelización básico

### Cluster 0: Clientes de valor medio con baja recencia
- **Características**: Baja recencia, frecuencia media, valor medio
- **Estrategia**: Reactivación personalizada
- **Acciones**:
  1. Desarrollar campaña "te extrañamos" con ofertas especiales
  2. Enviar recordatorios de productos previamente comprados
  3. Realizar encuesta para entender razones de inactividad

### Cluster 1: Clientes inactivos de bajo valor
- **Características**: Muy baja recencia, baja frecuencia, bajo valor
- **Estrategia**: Recuperación y reactivación
- **Acciones**:
  1. Lanzar campaña de "última oportunidad" con descuentos significativos
  2. Implementar programa de reactivación por etapas
  3. Evaluar ROI de marketing para este grupo

## Próximos Pasos

1. Implementar sistema de puntuación RFM continuo
2. Desarrollar campañas de marketing específicas por cluster
3. Establecer KPIs para medir éxito de estrategias
4. Realizar análisis de cohortes para seguimiento de evolución de clientes
5. Incorporar datos adicionales para segmentación más precisa

## Observaciones Finales de la Segmentación

- Clara segmentación en términos de valor y actividad del cliente
- Clusters 3 y 4 representan la mayoría de la base de clientes
- Oportunidad de mover clientes entre clusters adyacentes
- Urgencia en estrategias de reactivación para clusters 0 y 1

## Entrenamiento y pruebas de Modelos Predictivos

In [ ]:
# Preparar los datos
X = rfm.drop('Cluster', axis=1)
y = rfm['Cluster']

# Dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 1. Máquina de Soporte Vectorial (SVM)
svm = SVC(probability=True, random_state=42)
svm.fit(X_train, y_train)

# 2. K-Nearest Neighbors (KNN)
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

# 3. Árbol de Decisión
tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)

# 4. Gradient Boosting
gb = GradientBoostingClassifier(random_state=42)
gb.fit(X_train, y_train)

# 5. Naive Bayes
nb = GaussianNB()
nb.fit(X_train, y_train)

# 6. Ensamble de Votación
voting_clf = VotingClassifier(estimators=[
    ('svm', svm), 
    ('knn', knn), 
    ('tree', tree), 
    ('gb', gb), 
    ('nb', nb)
], voting='soft')
voting_clf.fit(X_train, y_train)

# 7. Redes Neuronales Artificiales (MLP)
mlp = MLPClassifier(random_state=42, max_iter=500)
mlp.fit(X_train, y_train)

# 8. Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

def imprimir_tabla_metricas(modelos, X_train, X_test, y_train, y_test):
    metricas = {}
    n_classes = len(np.unique(y_test))
    is_binary = n_classes == 2
    
    for nombre, modelo in modelos.items():
        # Medir tiempo de entrenamiento
        start_train = time.time()
        modelo.fit(X_train, y_train)
        end_train = time.time()
        train_time = end_train - start_train
        
        # Medir tiempo de prueba
        start_test = time.time()
        y_pred = modelo.predict(X_test)
        end_test = time.time()
        test_time = end_test - start_test
        
        if is_binary:
            y_prob = modelo.predict_proba(X_test)[:, 1]
            auc_roc = roc_auc_score(y_test, y_prob)
        else:
            y_prob = modelo.predict_proba(X_test)
            y_test_bin = label_binarize(y_test, classes=np.unique(y_test))
            auc_roc = roc_auc_score(y_test_bin, y_prob, multi_class='ovr', average='macro')
        
        metricas[nombre] = {
            'Precisión': precision_score(y_test, y_pred, average='macro'),
            'Sensibilidad': recall_score(y_test, y_pred, average='macro'),
            'F1-Score': f1_score(y_test, y_pred, average='macro'),
            'Accuracy': accuracy_score(y_test, y_pred),
            'AUC-ROC': auc_roc,
            'time_train': train_time,
            'time_test': test_time
        }
    
    # Crear tabla de métricas
    df_metricas = pd.DataFrame(metricas).T
    
    # Convertir las métricas de rendimiento a porcentaje
    metricas_porcentaje = ['Precisión', 'Sensibilidad', 'F1-Score', 'Accuracy', 'AUC-ROC']
    df_metricas[metricas_porcentaje] = df_metricas[metricas_porcentaje] * 100
    
    # Reordenar las columnas
    columnas_orden = ['Precisión', 'Sensibilidad', 'F1-Score', 'Accuracy', 'AUC-ROC', 'time_train', 'time_test']
    df_metricas = df_metricas[columnas_orden]

    # Función para formatear los valores sin ceros después del punto
    def format_value(value):
        if isinstance(value, float):
            return f"{value:.2f}".rstrip('0').rstrip('.')
        return value
    
    # Aplicar el formato a todas las celdas del DataFrame
    df_metricas = df_metricas.map(format_value)
    
    # Ordenar el DataFrame por la columna 'Precisión' de mayor a menor
    df_metricas_ordenado = df_metricas.sort_values(by='Precisión', ascending=False)
    
    # Mostrar la tabla ordenada usando display
    display(df_metricas_ordenado.style.set_caption("Tabla de Métricas (rendimiento en porcentaje, tiempos en segundos) - Ordenada por Precisión"))
    
    return df_metricas_ordenado

def graficar_metrica(df_metricas, metrica):
    # Convertir la columna seleccionada a tipo numérico
    df_metricas[metrica] = pd.to_numeric(df_metricas[metrica])
    
    # Ordenar el DataFrame por la métrica seleccionada
    df_sorted = df_metricas.sort_values(by=metrica, ascending=True)
    
    # Crear el gráfico
    plt.figure(figsize=(10, 6))
    ax = sns.barplot(x=df_sorted[metrica], y=df_sorted.index, palette='viridis')
    
    # Personalizar el gráfico
    plt.title(f'Comparación de Modelos - {metrica}')
    plt.xlabel('Valor de la Métrica')
    plt.ylabel('Modelo')
    
    # Ajustar el rango del eje x según la métrica
    if metrica in ['Precisión', 'Sensibilidad', 'F1-Score', 'Accuracy']:
        plt.xlim(80, 100)
    elif metrica == 'AUC-ROC':
        plt.xlim(95, 100)
    
    # Añadir etiquetas de valor en las barras
    for i, v in enumerate(df_sorted[metrica]):
        ax.text(v, i, f' {v}', va='center')
    
    plt.tight_layout()
    plt.show()

# Uso de las funciones
modelos = {
    'SVM': SVC(probability=True, random_state=42),
    'KNN': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'Naive Bayes': GaussianNB(),
    'Voting': VotingClassifier(estimators=[
        ('svm', SVC(probability=True, random_state=42)), 
        ('knn', KNeighborsClassifier()), 
        ('tree', DecisionTreeClassifier(random_state=42)), 
        ('gb', GradientBoostingClassifier(random_state=42)), 
        ('nb', GaussianNB())
    ], voting='soft'),
    'MLP': MLPClassifier(random_state=42, max_iter=500),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

# Imprimir tabla de métricas
df_metricas = imprimir_tabla_metricas(modelos, X_train, X_test, y_train, y_test)

# Graficar cada métrica
for metrica in ['Precisión', 'Sensibilidad', 'F1-Score', 'Accuracy', 'AUC-ROC']:
    graficar_metrica(df_metricas, metrica)

# Graficar tiempos de entrenamiento y prueba
graficar_metrica(df_metricas, 'time_train')
graficar_metrica(df_metricas, 'time_test')

# Análisis de Rendimiento de Modelos de Clasificación

## Resumen General
Los datos muestran el rendimiento de 8 modelos de clasificación diferentes, evaluados en 5 métricas distintas. Todos los modelos muestran un rendimiento generalmente bueno, con valores superiores al 80% en todas las métricas.

## Análisis por Modelo

### Random Forest
- **Mejor rendimiento general**
- Líder en todas las 5 métricas (Precisión, Sensibilidad, F1-Score, Accuracy, AUC-ROC)
- Rendimiento excepcionalmente equilibrado en todas las métricas
- Tiempo de entrenamiento moderado y muy rápido en prueba 

### Gradient Boosting
- Segundo mejor rendimiento general
- Muy cercano al Random Forest en todas las métricas
- Tiempo de entrenamiento largo, pero rápido en prueba 

### Voting (Ensamble de Votación)
- Tercer mejor rendimiento
- Muestra la fuerza de combinar múltiples modelos
- Excelente AUC-ROC (99.79%)
- Tiempo de entrenamiento más largo y rápido en prueba 

### Decision Tree (Árbol de Decisión)
- Sorprendentemente buen rendimiento para un modelo relativamente simple
- Cuarto en la mayoría de las métricas
- Menor AUC-ROC entre los modelos de alto rendimiento (97.13%)
- Extremadamente rápido en entrenamiento y prueba más rápida

### MLP (Red Neuronal)
- Rendimiento sólido pero no excepcional
- Consistente en todas las métricas
- Tiempo de entrenamiento moderado y muy rápido en prueba 

### Naive Bayes
- Rendimiento sorprendentemente bueno para un modelo tan simple
- Destaca en AUC-ROC (99.19%)
- El más rápido en entrenamiento y prueba

### KNN
- Rendimiento moderado
- Consistente en todas las métricas
- Muy rápido en entrenamiento y prueba 

### SVM
- Rendimiento más bajo en la mayoría de las métricas
- Aún así, muestra un buen AUC-ROC (97.29%)
- Tiempo de entrenamiento moderado y el más lento en prueba 

## Análisis por Métrica

1. **Precisión**: Random Forest lidera (97.41%), seguido de cerca por Gradient Boosting (96.95%).
2. **Sensibilidad**: Random Forest nuevamente en la cima (97.3%), con Gradient Boosting muy cerca (96.55%).
3. **F1-Score**: Random Forest mantiene el liderazgo (97.34%), seguido por Gradient Boosting (96.72%).
4. **Accuracy**: Random Forest continúa dominando (97.27%), con Gradient Boosting en segundo lugar (96.73%).
5. **AUC-ROC**: Todos los modelos muestran un excelente rendimiento, con Random Forest liderando (99.94%), seguido muy de cerca por Gradient Boosting (99.89%) y Voting (99.79%).

## Conclusiones

1. **Random Forest** se destaca como el mejor modelo general, liderando en todas las métricas con un buen equilibrio entre rendimiento y tiempo de ejecución.
2. **Gradient Boosting** es un fuerte competidor, muy cerca del rendimiento de Random Forest, pero con un tiempo de entrenamiento más largo.
3. Los modelos de **ensemble** (Random Forest, Gradient Boosting, Voting) muestran un rendimiento superior, lo que sugiere que la combinación de múltiples modelos es efectiva para este conjunto de datos.
4. Incluso los modelos más simples como **Decision Tree** y **Naive Bayes** muestran un rendimiento sorprendentemente bueno, especialmente considerando su velocidad.
5. El alto rendimiento en **AUC-ROC** para todos los modelos sugiere que son capaces de distinguir bien entre las clases, independientemente del umbral de clasificación elegido.

## Recomendaciones

1. **Utilizar Random Forest** como modelo principal debido a su rendimiento superior, consistente y buen equilibrio entre rendimiento y tiempo de ejecución.
2. Considerar **Gradient Boosting** como una alternativa sólida si el tiempo de entrenamiento no es una limitación crítica.
3. Si la velocidad de inferencia o la interpretabilidad son importantes, el **Decision Tree** podría ser una opción viable dado su buen rendimiento y rapidez extrema.
4. Para aplicaciones que requieran un balance entre rendimiento y simplicidad/velocidad, **Naive Bayes** podría ser una opción sorprendentemente efectiva.
5. Realizar un análisis de las características más importantes utilizadas por Random Forest y Gradient Boosting para obtener insights sobre el problema de clasificación.
6. Considerar el uso de **Voting** si se dispone de recursos computacionales suficientes, ya que ofrece un rendimiento muy alto, aunque con tiempos de ejecución más largos.